In [45]:
import json

ip_gold = "../predictions_extractions.json"
ip_kairos= "../event_role_KAIROS.json"

In [50]:
with open(ip_gold, 'r') as json_file:
    gold_data = json.load(json_file)
with open(ip_kairos, 'r') as json_file2:
    kairos_role = json.load(json_file2)    

In [47]:
kairos_role["ArtifactExistence.DamageDestroyDisableDismantle.Damage"]["template"]

'<arg1> damaged <arg2> using <arg3> instrument in <arg4> place'

In [51]:
total_role_types={}
for key,value in kairos_role.items():
    for i in range(len(value['roles'])):
        if(value['roles'][i] not in total_role_types.keys()):
            total_role_types[value['roles'][i]]=set()
        total_role_types[value['roles'][i]].update(value['role_types'][i])

In [52]:
print(len(total_role_types))

80


In [53]:
question_map = {}
for key, value in total_role_types.items():
    if ("per" in value or "org" in value):
        if("Who" not in question_map.keys()):
            question_map["Who"]=[]
        question_map["Who"].append(key)
    elif ("per" in value or "org" in value):
        if("Whom" not in question_map.keys()):
            question_map["Whom"]=[]
        question_map["Whom"].append(key)
    elif ("loc" in value or "fac" in value):
        if("Where" not in question_map.keys()):
            question_map["Where"]=[]
        question_map["Where"].append(key)
    elif ("mhi" in value):
        if("Why" not in question_map.keys()):
            question_map["Why"]=[]
        question_map["Why"].append(key)
    elif ("mhi" in value):
        if("How" not in question_map.keys()):
            question_map["How"]=[]
        question_map["How"].append(key)
    else:
        if("What" not in question_map.keys()):
            question_map["What"]=[]
        question_map["What"].append(key)
    

In [54]:
op_dictionary={}

In [55]:
def find_wh(role):
    wh_list=[]
    for k,v in question_map.items():
        if(role in v):
            wh_list.append(k) 
    return wh_list

In [57]:
#FOR PREDICTED EXATRACTIONS
for item in gold_data:
    op_dictionary[item["doc_id"]]={"events":[]}
    for event in item["event_mentions"]:
        temp_dict={}
        event_id=event["id"]
        trigger = event["trigger"]["text"]
        qa_list=[]
        for arg in event["arguments"]:
            arg_role=arg['role']
            arg_text=arg['text']
            wh = find_wh(arg_role)
            for wh_word in wh:
                question = wh_word + ' ' + trigger + '?'
                answer = arg_text
                qa_dict={'question': question,'answer': answer}
                qa_list.append(qa_dict)
        temp_dict[event_id]=qa_list
        op_dictionary[item["doc_id"]]["events"].append(temp_dict)

In [40]:
#For GOLD EXTRACTIONS
for key, value in gold_data.items():
    op_dictionary[key]={"events":[]}
    for item in value["event_mentions"]:
        temp_dict={}
        event_id=item["id"]
        trigger = item["trigger"]["text"]
        qa_list=[]
        for arg in item["arguments"]:
            arg_role=arg['role']
            arg_text=arg['text']
            wh = find_wh(arg_role)
            for wh_word in wh:
                question = wh_word + ' ' + trigger + '?'
                answer = arg_text
                qa_dict={'question': question,'answer': answer}
                qa_list.append(qa_dict)
        temp_dict[event_id]=qa_list
        op_dictionary[key]["events"].append(temp_dict)

In [59]:
pretty_json = json.dumps(op_dictionary, indent=4)
output_file_path = 'Rule_BasedQG_pred.json'
with open(output_file_path, 'w') as output_file:
    output_file.write(pretty_json)
print(f"Data saved to {output_file_path}")

Data saved to Rule_BasedQG_pred.json
